In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten ,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet152V2

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

import matplotlib.pyplot as plt
import os
import random

In [ ]:
%cd /content/drive/MyDrive/FYP/Data_set/train

!mkdir train
!mkdir test

train_percent = 0.8
test_percent = 0.1
val_percent = 0.1

import os
import random
from shutil import copyfile

classes = ["algal leaf", "Anthracnose", "bird eye spot", "brown blight", "gray light", "healthy", "red leaf spot", "white spot"]

for c in classes:
    os.makedirs("train/" + c)
    os.makedirs("test/" + c)
    os.makedirs("val/" + c)
    files = os.listdir(c)
    num_train = int(len(files) * train_percent)
    num_test = int(len(files) * test_percent)
    num_val = int(len(files) * val_percent)
    train_files = random.sample(files, num_train)
    test_val_files = list(set(files) - set(train_files))
    test_files = random.sample(test_val_files, num_test)
    val_files = list(set(test_val_files) - set(test_files))
    for f in train_files:
        copyfile(c + "/" + f, "train/" + c + "/" + f)
    for f in test_files:
        copyfile(c + "/" + f, "test/" + c + "/" + f)
    for f in val_files:
        copyfile(c + "/" + f, "val/" + c + "/" + f)

/content/drive/MyDrive/FYP/Data_set/train


In [ ]:
# Resize images
img_height = 224
img_width = 224
image_size = [img_height, img_width, 3] # height, width, depth

# Output classes
predict_class_size = 8

# batch size
bs = 32

# Train/test image path
train_data = '/content/drive/My Drive/FYP/Data_set/train/'
test_data = '/content/drive/My Drive/FYP/Data_set/val/'

In [ ]:
model_dict = {ResNet152V2:"resnet152v2"}

In [ ]:
def plot_loss_acc(model_history, model_name):
    
    ## Plot loss
    plt.plot(model_history.history['loss'], label='train loss')
    plt.plot(model_history.history['val_loss'], label='test loss')
    plt.legend()
    plt.savefig('/content/drive/My Drive/FYP/Data_set/model8/loss_{}'.format(model_name))
    plt.clf()

    ## Plot Accuracy
    plt.plot(model_history.history['accuracy'], label='train accuracy')
    plt.plot(model_history.history['val_accuracy'], label='test accuracy')
    plt.legend()
    plt.savefig('/content/drive/My Drive/FYP/Data_set/model8/accuracy_{}'.format(model_name))
    plt.clf()

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

def train_model(pre_trained_model):
    
    # Intialize pre trained model (ResNet152V2) with imagenet weights
    pre_trained_net = pre_trained_model(input_shape=image_size, weights='imagenet', include_top=False)

    # Unfreeze some of the layers for fine-tuning
    for layer in pre_trained_net.layers[-15:]:
        layer.trainable = True

    # Add new layers on top of the pre-trained model
    model = tf.keras.models.Sequential()

    model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(224,224,3)))
    model.add(MaxPooling2D())
    model.add(tf.keras.layers.Dropout(rate=0.3))

    model.add(Conv2D(32, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(tf.keras.layers.Dropout(rate=0.3))

    model.add(Conv2D(64, (3,3), 1, activation='relu'))
    model.add(MaxPooling2D())
    model.add(tf.keras.layers.Dropout(rate=0.3))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(Dense(8, activation='softmax'))    

    # Set cost and optimization functions for model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    # Image Data Augmentation using keras ImageDataGenerator
    # train data generator
    train_aug = ImageDataGenerator(rescale=1/255, 
                                   rotation_range=10,                         
                                   zoom_range=0.2, # ~ [1-0.2, 1+0.2]
                                   brightness_range=[0.5,1.5]) # <1 darkens img, >1 brightens img
    # test data generator
    test_aug = ImageDataGenerator(rescale=1/255)

    # Create train/test set
    train_set = train_aug.flow_from_directory(train_data, 
                                              target_size=(img_height,img_width), 
                                              batch_size=bs, 
                                              class_mode='categorical')
    

    test_set = test_aug.flow_from_directory(test_data, 
                                            target_size=(img_height,img_width), 
                                            batch_size=bs, 
                                            class_mode='categorical')   

    # fit the model
    hist = model.fit_generator(train_set, 
                              epochs=50,
                              steps_per_epoch=len(train_set),
                              validation_data=test_set,
                              validation_steps=len(test_set)
                             )
    
    # model name
    model_name = model_dict[pre_trained_model]
    
    # Save model loss/accuracy plots
    plot_loss_acc(hist,model_name)
    
    # Save model
    model.save('/content/drive/My Drive/FYP/Data_set/model8/model_{}.h5'.format(model_name))
    
    print("model {} trained and saved".format(model_name))

In [ ]:
for each_model in list(model_dict.keys()):
    train_model(each_model)

234545216/234545216 [==============================] - 9s 0us/step
Found 776 images belonging to 8 classes.
Found 80 images belonging to 8 classes.


<ipython-input-10-878c26066087>:60: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train_set,


Epoch 1/50
25/25 [==============================] - 693s 28s/step - loss: 2.4679 - accuracy: 0.1379 - val_loss: 2.0509 - val_accuracy: 0.3500
Epoch 2/50
25/25 [==============================] - 158s 6s/step - loss: 1.8445 - accuracy: 0.2719 - val_loss: 1.7595 - val_accuracy: 0.2625
Epoch 3/50
25/25 [==============================] - 147s 6s/step - loss: 1.5408 - accuracy: 0.3454 - val_loss: 1.6152 - val_accuracy: 0.2875
Epoch 4/50
25/25 [==============================] - 157s 6s/step - loss: 1.2895 - accuracy: 0.3943 - val_loss: 1.3131 - val_accuracy: 0.3750
Epoch 5/50
25/25 [==============================] - 147s 6s/step - loss: 1.1393 - accuracy: 0.4832 - val_loss: 1.3346 - val_accuracy: 0.3750
Epoch 6/50
25/25 [==============================] - 147s 6s/step - loss: 1.1095 - accuracy: 0.5219 - val_loss: 1.1126 - val_accuracy: 0.4625
Epoch 7/50
25/25 [==============================] - 149s 6s/step - loss: 1.0357 - accuracy: 0.5258 - val_loss: 1.0842 - val_accuracy: 0.6000
Epoch 8/50
2

<Figure size 640x480 with 0 Axes>

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = load_model('/content/drive/My Drive/FYP/Data_set/model8/model_resnet152v2.h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 111, 111, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 54, 54, 32)        0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
test_aug = ImageDataGenerator(rescale=1/255)

train_aug = ImageDataGenerator(rescale=1/255, 
                                   rotation_range=10,                         
                                   zoom_range=0.2, # ~ [1-0.2, 1+0.2]
                                   brightness_range=[0.5,1.5]) # <1 darkens img, >1 brightens img

In [ ]:
test_data = '/content/drive/My Drive/FYP/Data_set/val/'
train_data = '/content/drive/My Drive/FYP/Data_set/train/'

In [ ]:
img_height = 224
img_width = 224
image_size = [img_height, img_width, 3] # height, width, depth
# batch size
bs = 32

In [ ]:
test_set = test_aug.flow_from_directory(test_data, 
                                            target_size=(img_height,img_width), 
                                            batch_size=bs, 
                                            class_mode='categorical')   

train_set = train_aug.flow_from_directory(train_data, 
                                              target_size=(img_height,img_width), 
                                              batch_size=bs, 
                                              class_mode='categorical')
      


In [ ]:
# Predict the classes of the test set using the trained model
y_pred = model.predict(test_set)

# Get the class labels of the test set
y_true = test_set.classes

# Get the class names
class_names = list(train_set.class_indices.keys())

# Compute the confusion matrix
cm = confusion_matrix(y_true, np.argmax(y_pred, axis=1))

# Normalize the confusion matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# Plot the confusion matrix
plt.figure(figsize=(8, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

fmt = '.2f'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('/content/drive/My Drive/FYP/Data_set/model8/matrix'.format(confusion_matrix))







In [ ]:
from sklearn.metrics import f1_score


# Load the saved model
model = load_model('/content/drive/My Drive/FYP/Data_set/model8/model_resnet152v2.h5')


# Get predictions on test set
y_pred = model.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)

# Calculate F1 score
f1 = f1_score(test_set.classes, y_pred, average='weighted')

print('F1 score:', f1)



In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [ ]:
from sklearn.metrics import classification_report

# Predict classes for test set
y_pred = model.predict(test_set)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_set.classes

# Generate classification report
report = classification_report(y_true, y_pred_classes, target_names=list(test_set.class_indices.keys()))

print(report)
